In [ ]:
import keras

#importing MNIST data of 60000 examples and 784 features
(X_train, y_train), (X_test, y_test)=keras.datasets.mnist.load_data(path="/content/mnist.npz")

## Observations

- Importing MNIST dataset from keras datasets library into separate train and test variables for both features and labels.
- np.average method can be used to take averages of elements from different lists. 
- 
Experimental result:
=================================Implemented Function==========================================
Points in cluster 1: 8798
Points in cluster 2: 4616
Points in cluster 3: 4704
Points in cluster 4: 4176
Points in cluster 5: 5764
Points in cluster 6: 3015
Points in cluster 7: 5837
Points in cluster 8: 9973
Points in cluster 9: 4348
Points in cluster 10: 8769
=================================Library Function==========================================
4    7574
5    7232
8    7229
6    6895
9    5885
2    5465
1    5390
7    4946
3    4704
0    4680
dtype: int64


In [ ]:
import numpy as np
import pandas as pd 


X_train = np.reshape(X_train, (60000, 784))
X_test = np.reshape(X_test, (10000, 784))
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
X_train = X_train/255
X_test = X_test/255

In [ ]:
# import numpy as np
# import pandas as pd

# X_train = np.reshape(X_train, (60000, 784))
# X_train = pd.DataFrame(X_train)
# print(X_train.iloc[0].value_counts())

In [ ]:
# !pip install cython
# %load_ext Cython

In [ ]:
import numpy as np
import pandas as pd 
import keras

#importing MNIST data of 60000 examples and 784 features
(X_train, y_train), (X_test, y_test)=keras.datasets.mnist.load_data(path="/content/mnist.npz")

X_train = np.reshape(X_train, (60000, 784))
X_test = np.reshape(X_test, (10000, 784))
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
X_train = X_train/255
X_test = X_test/255

def MiniBatchKmeans(k: int, b: int, t: int, X: np.ndarray, random_state = None):

    shape_x = X.shape[0]    #shape of X along x-axis
    shape_y = X.shape[1]    #shape of X along y-axis

    #initializing centroids
    centroids = np.empty((0,shape_y))   #initially empty
    np.random.seed(random_state)    #seed random function with random state of the user
    x = np.random.choice(shape_x, size=k, replace=False)    #choose k numbers from [0,k) for indexing
    centroids = np.append(centroids, np.array(X[x]), axis=0)    #append chosen examples into centroids
    
    #count for each cluster
    v = np.zeros(k, dtype=int)

    #loop for max_iter iterations
    itr = 0
    flag = True
    while flag and itr < max_iter:
        flag = False
        chosen_indices = np.random.choice(shape_x, size = b, replace=False) #choose shape_x 
        M = X[chosen_indices]        #sample batch
        d = np.array([])    #distance matrix
        for i in range(k):
            sub = M - centroids[i]     #matrix - vector = all rows subtracted from matrix with corresponding vector values
            d = np.append(d, np.linalg.norm(sub, axis=1), axis=0)   #filling distance matrix
        itr += 1
        d=d.reshape((k, M.shape[0]))  #reshape d to (k, shape_x)
        for idx, x in enumerate(M):
            x_d = d[:,idx]
            c = np.where(x_d == min(x_d))[0][0]
            v[c] = v[c] + 1
            count = float(v[c])
            newc = np.array(((count - 1) * centroids[c] + x)/count)
            if not np.array_equal(newc, centroids[c]):
                centroids[c] = newc
                flag = True
    return centroids

k=10
b = 100
max_iter = 300
mbk = MiniBatchKmeans(k, b, max_iter, X_train)

In [ ]:
l1 = np.array([1,2,3])
l2 = np.array([1,2,3])
if np.array_equal(l1, l2):
    print(True)
else:
    print(False)

In [ ]:
%run -d -b27 minibatchkmeans.py 

In [ ]:
# a = np.array([[3, 1, 4],
#               [3, 2, 2],
#               [5, 3, 6],
#               [0,0,1]])
# # a = a/6
# min_a = np.amin(a, axis = 0)
# idxs = np.where(a == min_a)  # -1 is broadcast
# idxs = np.array(idxs)
# print(idxs[1])
# _, idx = np.unique(idxs[1], return_index=True)
# min_d = idxs[:,idx][0]
# # print(idxs)
# # print(np.unique(idxs[1]))
# c_idx = np.array(idxs[np.unique(idxs[:,1])])
# c_idx.transpose()

In [ ]:
# k = 10
# clusters = {}
# for i in range(k):
#     cluster[i] = np.array([])

# l = []

In [ ]:
#k-means 
import numpy as np
from scipy import sparse as sp
import pandas as pd

def allocate_clusters(X: pd.DataFrame, centroids: list, k: int):
  '''
  Reconstruct the clusters
  '''
  #reinitializing clusters
  clusters= {}
  for i in range(1,k+1):
    clusters[i] = []

  for i in range(X.shape[0]):
    x = X.loc[i]  #for every example x
    euc_dist = []  
    for j in range(1,k+1):
        euc_dist.append(np.linalg.norm(x - centroids[j]))  #find distances to all centroids from x
    clusters[euc_dist.index(min(euc_dist))+1].append([x])  #add x to the closest cluster
  return clusters

def update_centroids(X: pd.DataFrame, centroids: list, clusters: dict, k: int):
  '''
  Update centroid values by taking average 
  '''
  for i in range(1,k+1):
      centroids[i] = np.average(clusters[i], axis=0)[0] #find average for each feature
  return centroids


def Kmeans(X: pd.DataFrame, k: int, max_iter: int = 300, random_state: int = None):
  '''
  Implementation of K-Means clustering algorithm
  params:
  X: Sample Data
  k: Number of clusters
  max_iter (optional): Maximum number of iterations; default = 300
  random_state (optional): int or RandomState instance; default = None
  '''
  #validating if DataFrame
  if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X)

  #centering data for ease in distance computations
  if not sp.issparse(X):
            X_mean = X.mean(axis=0)
            # The copy was already done above
            X -= X_mean
  
  #initalizing centroids
  np.random.seed(random_state)
  centroids = [0]
  for i in range(1,k+1):
    x = np.random.randint(0,X.shape[0]-1)
    centroids.append(X.loc[x])

  #initializing clusters
  clusters= {}
  for i in range(1,k+1):
    clusters[i] = []

  #Step 1: Allocate examples to the closest clusters
  clusters = allocate_clusters(X, centroids, k)

  #loop until convergence
  flag = True   #flag to check convergence
  iter = 0
  while iter < max_iter and flag:
      flag = False #flag to check convergence

      #Step 1: Allocate examples to the closest clusters
      clusters = allocate_clusters(X, centroids, k)

      #Step 2: Update centroid values
      updated_centroids = update_centroids(X, centroids, clusters, k)
      
      #check for convergence
      if not update_centroids == centroids:
          flag = True

      centroids = updated_centroids   #updated centroids

      iter += 1   #increment iteration count
      
  #print output
  for i in range(1,k+1):
    print(f'Points in cluster {i}: {len(clusters[i])}')
  return centroids

In [ ]:
# #initializing clusters
# clusters= {}
# for i in range(k):
#     clusters[i] = []


# for i in range(k):


In [ ]:
from sklearn.metrics import pairwise_distances_argmin
from sklearn.cluster import MiniBatchKMeans



b = 100
max_iter = 1000
for k in range(1,10):
    mbki = MiniBatchKmeans(k, b, max_iter, X_train)

    labelsi = pairwise_distances_argmin(X_train, mbki)
    print(pd.Series(labelsi).value_counts())

for k in range(10, 1000, 10):
    mbki = MiniBatchKmeans(k, b, max_iter, X_train)

    labelsi = pairwise_distances_argmin(X_train, mbki)
    print(pd.Series(labelsi).value_counts())



In [ ]:
# 9    9546
# 2    8711
# 6    6520
# 5    5986
# 1    5970
# 3    5689
# 4    5267
# 0    4746
# 8    4041
# 7    3524
# dtype: int64

In [ ]:
# print(__doc__)

# import time

# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn.cluster import MiniBatchKMeans, KMeans
# from sklearn.metrics.pairwise import pairwise_distances_argmin
# from sklearn.datasets import make_blobs

# # #############################################################################
# # Generate sample data
# np.random.seed(0)

# batch_size = 45
# centers = [[1, 1], [-1, -1], [1, -1]]
# n_clusters = len(centers)
# X, labels_true = make_blobs(n_samples=3000, centers=centers, cluster_std=0.7)

# # #############################################################################
# # Compute clustering with Means

# k_means = KMeans(init='k-means++', n_clusters=3, n_init=10)
# t0 = time.time()
# k_means.fit(X)
# t_batch = time.time() - t0

# # #############################################################################
# # Compute clustering with MiniBatchKMeans

# mbk = MiniBatchKMeans(init='k-means++', n_clusters=3, batch_size=batch_size,
#                       n_init=10, max_no_improvement=10, verbose=0)
# t0 = time.time()
# mbk.fit(X)
# t_mini_batch = time.time() - t0

# # #############################################################################
# # Plot result

# fig = plt.figure(figsize=(8, 3))
# fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
# colors = ['#4EACC5', '#FF9C34', '#4E9A06']

# # We want to have the same colors for the same cluster from the
# # MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
# # closest one.
# k_means_cluster_centers = k_means.cluster_centers_
# order = pairwise_distances_argmin(k_means.cluster_centers_,
#                                   mbk.cluster_centers_)
# mbk_means_cluster_centers = mbk.cluster_centers_[order]
# print(k_means_cluster_centers)
# k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
# mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)
# print(k_means_labels)
# # KMeans
# ax = fig.add_subplot(1, 3, 1)
# for k, col in zip(range(n_clusters), colors):
#     my_members = k_means_labels == k
#     cluster_center = k_means_cluster_centers[k]
#     ax.plot(X[my_members, 0], X[my_members, 1], 'w',
#             markerfacecolor=col, marker='.')
#     ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
#             markeredgecolor='k', markersize=6)
# ax.set_title('KMeans')
# ax.set_xticks(())
# ax.set_yticks(())
# plt.text(-3.5, 1.8,  'train time: %.2fs\ninertia: %f' % (
#     t_batch, k_means.inertia_))

# # MiniBatchKMeans
# ax = fig.add_subplot(1, 3, 2)
# for k, col in zip(range(n_clusters), colors):
#     my_members = mbk_means_labels == k
#     cluster_center = mbk_means_cluster_centers[k]
#     ax.plot(X[my_members, 0], X[my_members, 1], 'w',
#             markerfacecolor=col, marker='.')
#     ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
#             markeredgecolor='k', markersize=6)
# ax.set_title('MiniBatchKMeans')
# ax.set_xticks(())
# ax.set_yticks(())
# plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' %
#          (t_mini_batch, mbk.inertia_))

# # Initialise the different array to all False
# different = (mbk_means_labels == 4)
# ax = fig.add_subplot(1, 3, 3)

# for k in range(n_clusters):
#     different += ((k_means_labels == k) != (mbk_means_labels == k))

# identic = np.logical_not(different)
# ax.plot(X[identic, 0], X[identic, 1], 'w',
#         markerfacecolor='#bbbbbb', marker='.')
# ax.plot(X[different, 0], X[different, 1], 'w',
#         markerfacecolor='m', marker='.')
# ax.set_title('Difference')
# ax.set_xticks(())
# ax.set_yticks(())

# # plt.show()

In [ ]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans

In [ ]:
# # reading the data and looking at the first five rows of the data
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data)

In [ ]:

print('=================================Implemented Function==========================================')
centroids = Kmeans(X_train, 10, 15, random_state=42)

# # k means using 10 clusters initialization
kmeans = KMeans(n_clusters = 10, max_iter = 15, random_state=42)

kmeans.fit(X_train)
pred = kmeans.labels_
print('=================================Library Function==========================================')
print(pd.Series(pred).value_counts())

In [ ]:
print(pd.Series(y_train).value_counts())

In [ ]:
# cluster = [-0.65607391, 0.50791595,  0.14838137, -0.49656661, -0.56620088, -0.25608672, -0.50849408, -0.3223056 ]
# average = np.average(cluster, axis=0)
# if not np.abs(average - cluster).any():
#   flag = True
# cluster = average
# print(flag)

In [ ]:
# def bar():
#   x = 0
#   foo()
# def foo():
#   print(x)

# bar()

In [ ]:
'''
References:
  Gaurav Sir's Notes and Lectures
  https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-k-means-clustering/
  https://docs.python.org/3/library/random.html
  https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions
  https://www.geeksforgeeks.org/calculate-the-euclidean-distance-using-numpy/
  https://www.geeksforgeeks.org/python-maximum-minimum-elements-position-list/
  https://towardsdatascience.com/k-means-without-libraries-python-feb3572e2eef
  https://stackoverflow.com/questions/14808945/check-if-variable-is-dataframe
  https://stackoverflow.com/questions/14808945/check-if-variable-is-dataframe
  https://blog.finxter.com/compare-two-lists-in-python/#:~:text=Short%20answer%3A%20The%20most%20Pythonic,the%20return%20value%20is%20True%20.
  https://www.geeksforgeeks.org/random-seed-in-python/
  https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
  https://stackoverflow.com/questions/58842373/typeerror-int-object-is-not-callable-in-np-random-seed
  https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/cluster/_kmeans.py#L749
  https://numpy.org/doc/stable/reference/generated/numpy.append.html
  https://www.geeksforgeeks.org/how-to-randomly-select-rows-from-pandas-dataframe/
  https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html
  https://stackoverflow.com/questions/26333005/numpy-subtract-every-row-of-matrix-by-vector/26333184
  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html
  https://numpy.org/doc/stable/reference/generated/numpy.divide.html
  https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.06-Errors-and-Debugging.ipynb#scrollTo=xqgyKJQbqNOy
  https://numpy.org/doc/stable/reference/random/generated/numpy.random.shuffle.html
  https://stackoverflow.com/questions/6980749/simpler-way-to-put-pdb-breakpoints-in-python-code
  https://stackoverflow.com/questions/23911875/select-certain-rows-condition-met-but-only-some-columns-in-python-numpy
  https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html#sphx-glr-auto-examples-cluster-plot-mini-batch-kmeans-py
  https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances_argmin.html#sklearn.metrics.pairwise_distances_argmin
'''

In [ ]:
'''
Why algo is so slow?
 https://scikit-learn.org/stable/modules/clustering.html#k-means
Use of MiniBatchKMeans to reduce computations: https://medium.com/datadriveninvestor/k-means-clustering-for-imagery-analysis-56c9976f16b6
Streaming average: https://blog.superfeedr.com/streaming-average/
Algo for MiniBatchKmeans : https://www.eecs.tufts.edu/~dsculley/papers/fastkmeans.pdf
Difference between lists and numpy array: https://www.geeksforgeeks.org/python-lists-vs-numpy-arrays/
Taking min across columns of numpy array: https://thispointer.com/numpy-amin-find-minimum-value-in-numpy-array-and-its-index/
'''
